In [1]:
# ## KAGGLE ONLY
# from shutil import copyfile
# copyfile(src="../input/inputs/generate_dataloaders.py", dst="../working/generate_dataloaders.py")
# copyfile(src="../input/inputs/train_dataloader.p", dst="../working/train_dataloader.p")
# copyfile(src="../input/inputs/val_dataloader.p", dst="../working/val_dataloader.p")
# copyfile(src="../input/inputs/centroids_dataloader.p", dst="../working/ground_truth_dataloader.p")
# copyfile(src="../input/inputs/dictionary.p", dst="../working/dictionary.p")

# copyfile(src="../input/input2/train_unlabeld_dataloader.p", dst="../working/train_unlabelled_dataloader.p")
# copyfile(src="../input/input2/train_labeled_dataloader.p", dst="../working/train_labelled_dataloader.p")

In [1]:
import numpy as np
import matplotlib.pyplot as plt

import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader
import torch.nn.functional as F

#from datasets import get_mnist_dataset, get_data_loader
#from utils import *
#from models import *

import pickle as pkl
import os
import datetime as dt
import pandas as pd
import random

from generate_dataloaders import *

from tqdm import tqdm_notebook as tqdm

import evaluation
import importlib
importlib.reload(evaluation)

<module 'evaluation' from '/Users/elliotsilva/Desktop/DS-GA-1006/FairFrame/evaluation.py'>

## Get Dataloaders

In [2]:
seed = 1029
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)  # if you are using multi-GPU.
np.random.seed(seed)  # Numpy module.
random.seed(seed)  # Python random module.
torch.manual_seed(seed)
torch.backends.cudnn.enabled = False 
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

if torch.cuda.is_available(): torch.cuda.manual_seed_all(seed)

def _init_fn(worker_id):
    np.random.seed(int(seed))

In [3]:
path = os.getcwd()
data_dir = path + '/'
data_dir = path +'/data/' #Uncomment for local system

#### *Verify filenames are consistent*

In [4]:
train_loader_labelled = pkl.load(open(data_dir + 'train_labeled_dataloader.p','rb'))
train_loader_unlabelled = pkl.load(open(data_dir + 'train_unlabeled_dataloader.p','rb'))
val_loader = pkl.load(open(data_dir + 'val_dataloader.p','rb'))

In [5]:
review_dict = pkl.load(open(data_dir + 'dictionary.p','rb'))

In [6]:
#%conda install pytorch torchvision -c pytorch
## if torch.__version__ is not 1.3.1, run this cell then restart kernel

In [7]:
print(torch.__version__)

1.3.1


## PRE TRAINED WORD EMBEDDINGS 

In [8]:
def get_coefs(word, *arr):
    return word, np.asarray(arr, dtype='float16')

In [9]:
def load_embeddings(path):
    with open(path) as f:
        return dict(get_coefs(*line.strip().split(' ')) for line in tqdm(f))

In [10]:
def build_matrix(review_dict, embedding_index ,dim = 200):
#     embedding_index = load_embeddings(path)
    embedding_matrix = np.zeros((len(review_dict.tokens), dim))
    unknown_words = []
    
    for word, i in review_dict.ids.items():
        try:
            embedding_matrix[i] = embedding_index[word]
        except KeyError:
            unknown_words.append(word)
    return embedding_matrix, unknown_words

In [11]:
## LOCAL - 2nd line // KAGGLE -- 1st line
# glove_twitter = '../input/glove-global-vectors-for-word-representation/glove.twitter.27B.200d.txt' #Change loc for local system
glove_twitter = data_dir + 'glove.twitter.27B.200d.txt'

In [12]:
embedding_index = load_embeddings(glove_twitter)

In [13]:
glove_embedding_index,unknown_words = build_matrix(review_dict, embedding_index)
del embedding_index

In [14]:
len(review_dict.tokens)

16256

In [15]:
len(unknown_words)

4428

In [16]:
# for word in unknown_words:
#     print(word)

## Neural Network LSTM Class

NOTE: Data loader is defined as:
- tuple: (tokens, flagged_index, problematic)

In [17]:
class LSTM_model(nn.Module):
    """
    LSTM classification model using pretrained glove embeddings
    """
    # NOTE: we can't use linear layer until we take weighted average, otherwise it will
    # remember certain positions incorrectly (ie, 4th word has bigger weights vs 7th word)
    def __init__(self, embedding_matrix, num_hidden_layers = 3, hidden_size = 100, num_classes = 2):
        super(LSTM_model, self).__init__()
        vocab_size = embedding_matrix.shape[0]
        embed_size = embedding_matrix.shape[1]
        self.embedding_matrix = embedding_matrix
        
        self.vocab_size = vocab_size
        self.embed_size = embed_size
        self.num_hidden_layers = num_hidden_layers
        self.hidden_size = hidden_size
        self.num_classes = num_classes
        
        self.embed = nn.Embedding(vocab_size, embed_size, padding_idx=0)    
        self.embed.weight = nn.Parameter(torch.tensor(embedding_matrix, dtype=torch.float32))
        self.embed.weight.requires_grad = False
        
        self.lstm = nn.LSTM(self.embed_size,self.hidden_size,self.num_hidden_layers, batch_first=True,bidirectional= True,bias=True)
        
        self.projection = nn.Linear(2*self.hidden_size, self.num_classes, bias=True)

    
    def forward(self, tokens, flagged_index):
        batch_size, num_tokens = tokens.shape
        embedding = self.embed(tokens)
#         print(embedding.shape) # below assumes "batch_size x num_tokens x Emb_dim" (VERIFY)
        
        lstm_output = self.lstm(embedding)
        # lstm_output is a tuple containing lstm output and (hidden_state, lstm_cell). 
        # lstm_output[0] would be of shape "batch_size x num_tokens x hidden_size" (VERIFY)
        
        logits = self.projection(lstm_output[0])
        # logits would be of shape "batch_size x num_tokens x num_classes (2)" (VERIFY)
        
        batch_size, _, __ = logits.shape
        
        #selecting the logit at the flagged index
        relevant_logits = logits[list(range(batch_size)),flagged_index]
        # relevant_logits would be of shape "batch_size x num_classes (2)" (VERIFY)
        
        return relevant_logits

## First performing fully supervised learning using the labelled set to train new vector representations

In [18]:
num_gpus = torch.cuda.device_count()
if num_gpus > 0:
    current_device = 'cuda'
else:
    current_device = 'cpu'

model = LSTM_model(glove_embedding_index, num_hidden_layers = 3, hidden_size = 100, num_classes = 2).to(current_device)

In [19]:
criterion = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)

### Scratch work to understand the lstm output and the loss function

In [21]:
for i,X in enumerate(val_loader):
    tokens_labelled, labels, flagged_indices_labelled, = X
    break

In [22]:
tokens_labelled = tokens_labelled.to(current_device)
flagged_indices_labelled = flagged_indices_labelled.to(current_device)
labels = labels.to(current_device)

In [23]:
tokens_labelled.shape

torch.Size([32, 30])

In [24]:
embeddings = model.embed(tokens_labelled)
embeddings.shape

torch.Size([32, 30, 200])

In [25]:
lstm_output = model.lstm(embeddings)
lstm_output[0].shape

torch.Size([32, 30, 200])

In [26]:
logits = model.projection(lstm_output[0])
logits.shape

torch.Size([32, 30, 2])

In [27]:
relevant_logits = logits[list(range(32)),flagged_indices_labelled]
relevant_logits.shape

torch.Size([32, 2])

In [28]:
labels.shape

torch.Size([32])

In [29]:
loss = criterion(relevant_logits,labels)
loss

tensor(21.9358, grad_fn=<NllLossBackward>)

## Supervised model training

In [20]:
def train_supervised_model(model, criterion, optimizer, train_loader_labelled, valid_loader, num_epochs=10, path_to_save=None, print_every = 1000):

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    for epoch in range(num_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.train()
        total_epoch_loss = 0
        
        for i,(tokens_labelled, labels, flagged_indices_labelled) in tqdm(enumerate(train_loader_labelled)):
            
            tokens_labelled = tokens_labelled.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            labels = labels.to(current_device)

            # forward pass and compute loss
            logits = model(tokens_labelled,flagged_indices_labelled)
            
            loss = criterion(logits, labels)
        
            # run update step
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            #Add loss to the epoch loss
            total_epoch_loss += loss

            if i % print_every == 0:
                losses = loss/len(tokens_labelled)
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= len(train_loader_labelled.dataset)
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            logits = model(tokens,flagged_indices)
            
            loss = criterion(logits, labels)
            
            #Add loss to the validation loss
            total_validation_loss += loss

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix,\
                    "num_hidden_layers":model.num_hidden_layers,\
                    "hidden_size":model.hidden_size,\
                    "num_classes":model.num_classes}
            torch.save(model.state_dict(), path_to_save+'model_dict_labelled.pt')
            torch.save(train_losses, path_to_save+'train_losses_labelled')
            torch.save(val_losses, path_to_save+'val_losses_labelled')
            torch.save(opts, path_to_save+'opts_labelled')
        
    return model, train_losses, val_losses

In [56]:
path = os.getcwd()
model_folder = 'lstm_model/'
model_dir = path + '/models/' + model_folder

In [57]:
len(train_loader_labelled)

114

In [58]:
lstm_model, train_losses, val_losses = train_supervised_model(model, criterion, optimizer, train_loader_labelled, val_loader, num_epochs=10, path_to_save=model_dir)


2019-11-22 16:51:50.051952 | Epoch 0


Average training loss at batch  0 : 0.005

Average training loss after epoch  0 : 0.067
Average validation loss after epoch  0 : 0.367
2019-11-22 16:52:05.570541 | Epoch 1


Average training loss at batch  0 : 0.037

Average training loss after epoch  1 : 0.061
Average validation loss after epoch  1 : 0.355
2019-11-22 16:52:19.768974 | Epoch 2


Average training loss at batch  0 : 0.015

Average training loss after epoch  2 : 0.062
Average validation loss after epoch  2 : 0.353
2019-11-22 16:52:34.126721 | Epoch 3


Average training loss at batch  0 : 0.009

Average training loss after epoch  3 : 0.046
Average validation loss after epoch  3 : 0.419
2019-11-22 16:52:48.495477 | Epoch 4


...

Average training loss at batch  0 : 0.027

Average training loss after epoch  5 : 0.051
Average validation loss after epoch  5 : 0.398
2019-11-22 16:53:16.426880 | Epoch 6


Average training loss at batch  0 : 0.033

Average training loss after epoch  6 : 0.047
Average validation loss after epoch  6 : 0.458
2019-11-22 16:53:30.168134 | Epoch 7


Average training loss at batch  0 : 0.113

Average training loss after epoch  7 : 0.036
Average validation loss after epoch  7 : 0.385
2019-11-22 16:53:43.629549 | Epoch 8


Average training loss at batch  0 : 0.017

Average training loss after epoch  8 : 0.025
Average validation loss after epoch  8 : 0.552
2019-11-22 16:53:58.035510 | Epoch 9


Average training loss at batch  0 : 0.015

Average training loss after epoch  9 : 0.020
Average validation loss after epoch  9 : 0.495


### Clustering Stuff

In [21]:
class KMeansCriterion(nn.Module):
    
    def __init__(self, lmbda):
        super().__init__()
        self.lmbda = lmbda
    
    def forward(self, embeddings, centroids, labelled = False,  cluster_assignments = None):
        if labelled:
            num_reviews = len(cluster_assignments)
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances = distances[list(range(num_reviews)),cluster_assignments]
            loss = self.lmbda * cluster_distances.sum()
        else:
            distances = torch.sum((embeddings[:, None, :] - centroids)**2, 2)
            cluster_distances, cluster_assignments = distances.min(1)
            loss = self.lmbda * cluster_distances.sum()
        return loss, cluster_assignments

In [22]:
def centroid_init(k, d, dataloader, model, current_device):
    ## Here we ideally don't want to do randomized/zero initialization
    centroid_sums = torch.zeros(k, d).to(current_device)
    centroid_counts = torch.zeros(k).to(current_device)
    for (tokens, labels, flagged_indices) in dataloader:
        # cluster_assignments = torch.LongTensor(tokens.size(0)).random_(k)
        cluster_assignments = labels.to(current_device)
        
        model.eval()
        sentence_embed = model(tokens.to(current_device),flagged_indices.to(current_device))
    
        update_clusters(centroid_sums, centroid_counts,
                        cluster_assignments, sentence_embed.to(current_device))
    
    centroid_means = centroid_sums / centroid_counts[:, None].to(current_device)
    return centroid_means.clone()

def update_clusters(centroid_sums, centroid_counts,
                    cluster_assignments, embeddings):
    k = centroid_sums.size(0)

    centroid_sums.index_add_(0, cluster_assignments, embeddings)
    bin_counts = torch.bincount(cluster_assignments,minlength=k).type(torch.FloatTensor).to(current_device)
    centroid_counts.add_(bin_counts)

## Dataloader stuff

In [23]:
def loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled):
    try:
        tokens, labels, flagged_indices = next(train_loader_labelled_iter)
    except StopIteration:
        train_loader_labelled_iter = iter(train_loader_labelled)
        tokens, labels, flagged_indices = next(train_loader_labelled_iter)

    return tokens, labels, flagged_indices, train_loader_labelled_iter


def loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled):
    try:
        tokens, labels, flagged_indices = next(train_loader_unlabelled_iter)
    except StopIteration:
        train_loader_unlabelled_iter = iter(train_loader_unlabelled)
        tokens, labels, flagged_indices = next(train_loader_unlabelled_iter)

    return tokens, labels, flagged_indices, train_loader_unlabelled_iter

### Training Function

In [38]:
def train_clusters(model, centroids, criterion, optimizer, train_loader_labelled, train_loader_unlabelled, valid_loader, num_epochs=10, num_batches = 1000, path_to_save=None, print_every = 1000):

    train_loader_labelled_iter = iter(train_loader_labelled)
    train_loader_unlabelled_iter = iter(train_loader_unlabelled)

    train_losses=[]
    val_losses=[]
    num_gpus = torch.cuda.device_count()
    if num_gpus > 0:
        current_device = 'cuda'
    else:
        current_device = 'cpu'
    
    for epoch in range(num_epochs):
        print('{} | Epoch {}'.format(dt.datetime.now(), epoch))
        model.eval() # we're only clustering, not training model
        k, d = centroids.size()
        centroid_sums = torch.zeros_like(centroids).to(current_device)
        centroid_counts = torch.zeros(k).to(current_device)
        total_epoch_loss = 0
        
        for i in tqdm(range(int(num_batches))):
            tokens_labelled, labels, flagged_indices_labelled, train_loader_labelled_iter = loadLabelledBatch(train_loader_labelled_iter, train_loader_labelled)
            tokens_unlabelled, _, flagged_indices_unlabelled, train_loader_unlabelled_iter = loadUnlabelledBatch(train_loader_unlabelled_iter, train_loader_unlabelled)

            tokens_labelled = tokens_labelled.to(current_device)
            labels = labels.to(current_device)
            flagged_indices_labelled = flagged_indices_labelled.to(current_device)
            
            tokens_unlabelled = tokens_unlabelled.to(current_device)
            flagged_indices_unlabelled = flagged_indices_unlabelled.to(current_device)

            # forward pass and compute loss
            sentence_embed_labelled = model(tokens_labelled,flagged_indices_labelled)
            sentence_embed_unlabelled = model(tokens_unlabelled,flagged_indices_unlabelled)
            
            cluster_loss_unlabelled, cluster_assignments_unlabelled = criterion(sentence_embed_unlabelled, centroids.detach())
            cluster_loss_labelled, cluster_assignments_labelled = criterion(sentence_embed_labelled, centroids.detach(), labelled = True, cluster_assignments = labels)
    
            total_batch_loss = cluster_loss_labelled + cluster_loss_unlabelled
            
#             #Add loss to the epoch loss
            total_epoch_loss += total_batch_loss

#             # store centroid sums and counts in memory for later centering
            update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                            cluster_assignments_labelled.detach(), sentence_embed_labelled.detach())
    
            update_clusters(centroid_sums.detach(), centroid_counts.detach(),
                            cluster_assignments_unlabelled.detach(), sentence_embed_unlabelled.detach())

            if i % print_every == 0:
                losses = total_batch_loss/(len(tokens_labelled)+ len(tokens_unlabelled))
                print('Average training loss at batch ',i,': %.3f' % losses)
            
        total_epoch_loss /= (len(train_loader_labelled.dataset)+len(train_loader_unlabelled.dataset))
        train_losses.append(total_epoch_loss)
        print('Average training loss after epoch ',epoch,': %.3f' % total_epoch_loss)
        
        # update centroids based on assignments from autoencoders
        centroids = centroid_sums / (centroid_counts[:, None] + 1).to(current_device)
        
        # calculate validation loss after every epoch
        total_validation_loss = 0
        for i, (tokens, labels, flagged_indices) in enumerate(valid_loader):
            model.eval()
            tokens = tokens.to(current_device)
            labels = labels.to(current_device)
            flagged_indices = flagged_indices.to(current_device)
            
            # forward pass and compute loss
            sentence_embed = model(tokens,flagged_indices)
            cluster_loss, cluster_assignments = criterion(sentence_embed, centroids)
            
            #Add loss to the validation loss
            total_validation_loss += cluster_loss.data

        total_validation_loss /= len(valid_loader.dataset)
        val_losses.append(total_validation_loss)
        print('Average validation loss after epoch ',epoch,': %.3f' % total_validation_loss)
        
        if path_to_save == None:
            pass
        else:
            opts = {"embedding_matrix":model.embedding_matrix,\
                    "num_hidden_layers":model.num_hidden_layers,\
                    "hidden_size":model.hidden_size,\
                    "num_classes":model.num_classes}
            torch.save(model.state_dict(), path_to_save+'model_dict_unlabelled.pt')
            torch.save(train_losses, path_to_save+'train_losses_unlabelled')
            torch.save(val_losses, path_to_save+'val_losses_unlabelled')
            torch.save(opts, path_to_save+'opts_unlabelled')
            
        
    return model, centroids, train_losses, val_losses

In [39]:
unsupervised_model = model
unsupervised_model.projection = nn.Identity()

In [40]:
centroids = centroid_init(2, 2*unsupervised_model.hidden_size, train_loader_labelled, unsupervised_model, current_device)
criterion = KMeansCriterion(1).to(current_device)
optimizer = torch.optim.Adam(model.parameters(), 0.01, amsgrad=True)

In [41]:
centroids.shape

torch.Size([2, 200])

In [42]:
path = os.getcwd()
model_folder = 'lstm_model/'
model_dir = path + '/models/' + model_folder

In [43]:
num_batches = int(len(train_loader_unlabelled.dataset)/train_loader_unlabelled.batch_size)+1
num_batches

3211

In [44]:
lstm_model, lstm_centroids, lstm_train_losses, lstm_val_losses = train_clusters(unsupervised_model, centroids, criterion, optimizer, train_loader_labelled,train_loader_unlabelled, val_loader, num_epochs=10, num_batches=num_batches, path_to_save=model_dir)


2019-11-22 17:34:58.615589 | Epoch 0


Average training loss at batch  0 : 0.025

Average training loss after epoch  0 : 0.005
Average validation loss after epoch  0 : 0.018
2019-11-22 17:35:43.874633 | Epoch 1


Average training loss at batch  0 : 0.024

Average training loss after epoch  1 : 0.005
Average validation loss after epoch  1 : 0.018
2019-11-22 17:37:16.261650 | Epoch 2


Average training loss at batch  0 : 0.024

Average training loss after epoch  2 : 0.005
Average validation loss after epoch  2 : 0.018
2019-11-22 17:39:05.671874 | Epoch 3


Average training loss at batch  0 : 0.023

Average training loss after epoch  3 : 0.005
Average validation loss after epoch  3 : 0.018
2019-11-22 17:40:56.352126 | Epoch 4


...

Average training loss at batch  0 : 0.025

Average training loss after epoch  5 : 0.005
Average validation loss after epoch  5 : 0.018
2019-11-22 17:45:08.238891 | Epoch 6


Average training loss at batch  0 : 0.026

Average training loss after epoch  6 : 0.005
Average validation loss after epoch  6 : 0.018
2019-11-22 17:47:08.386033 | Epoch 7


Average training loss at batch  0 : 0.023

Average training loss after epoch  7 : 0.005
Average validation loss after epoch  7 : 0.018
2019-11-22 17:49:12.584786 | Epoch 8


Average training loss at batch  0 : 0.023

Average training loss after epoch  8 : 0.005
Average validation loss after epoch  8 : 0.018
2019-11-22 17:51:23.835066 | Epoch 9


Average training loss at batch  0 : 0.023

Average training loss after epoch  9 : 0.005
Average validation loss after epoch  9 : 0.018


In [ ]:
# #Only needed for Kaggle

# from IPython.display import FileLink, FileLinks 
# FileLinks('.') #lists all downloadable files on server